# Information Retrieval Project

Bagging Model for Product Title Quality with Noise

CIKM AnalytiCup 2017

Aarib Ahmed Vahidy

Partham Kumar Chawla

### Loading Libraries

In [1]:
pip install pandas numpy scikit-learn xgboost lightgbm nltk gensim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\aarib\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
import xgboost as xgb
import warnings
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

warnings.filterwarnings('ignore')

### Data Loading and Label Integration

Training and validation datasets are loaded from CSV files.

Corresponding labels (clarity and conciseness) are loaded from separate .labels and .predict files.

These labels are assigned meaningful column names.

The label files are then concatenated with their respective main datasets.

Column names of both train and validation DataFrames are standardized for consistency and easier downstream processing.

### Loading training data

In [3]:
#Loading the main data
data_train = pd.read_csv(r'C:\Users\aarib\6thSemester\IR\IRProject\Product Title Classification\CIKMAnalytiCup2017_Lazada\training\data_train.csv')

#Loading labels
clarity_labels = pd.read_csv(r'C:\Users\aarib\6thSemester\IR\IRProject\Product Title Classification\CIKMAnalytiCup2017_Lazada\training\clarity_train.labels', header=None)
conciseness_labels = pd.read_csv(r'C:\Users\aarib\6thSemester\IR\IRProject\Product Title Classification\CIKMAnalytiCup2017_Lazada\training\conciseness_train.labels', header=None)

#Assigning meaningful column names
clarity_labels.columns = ['clarity']
conciseness_labels.columns = ['conciseness']

#Combining everything into one DataFrame
train = pd.concat([data_train, clarity_labels, conciseness_labels], axis=1)

print("Shape of training data:", train.shape)
train.head()

Shape of training data: (36283, 11)


,my,AD674FAASTLXANMY,Adana Gallery Suri Square Hijab – Light Pink,Fashion,Women,Muslim Wear,<ul><li>Material : Non sheer shimmer chiffon</li><li>Sizes : 52 x 52 inches OR 56 x 56 inches</li><li>Cut with curved ends</li></ul>,49.0,local,clarity,conciseness
0,my,AE068HBAA3RPRDANMY,Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz,Health & Beauty,Bath & Body,Hand & Foot Care,Formulated with oil-free hydrating botanicals/...,128.00,international,1,1
1,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,<ul> <li>150cm mini microphone compatible for ...,25.07,international,1,1
2,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,<ul> <li>ANMYNA Complaint Silky Set (Shampoo 5...,118.00,local,1,0
3,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,<ul> <li>100% Authentic</li> <li>Rrefresh and ...,114.80,international,1,1
4,my,AS575ELCMZ4WANMY,Asus TP300LJ-DW004H Transformer Book Flip 4GB ...,Computers & Laptops,Laptops,Traditional Laptops,"<div class=""prod_content""> <div class=""prod_de...",2599.00,local,1,1


### Adding column names to training data

In [4]:
train.columns = [
    'country', 'product_id', 'title', 
    'category_lvl_1', 'category_lvl_2', 'category_lvl_3',
    'description', 'price', 'delivery_type', 
    'clarity', 'conciseness'
]

In [5]:
train.head()

,country,product_id,title,category_lvl_1,category_lvl_2,category_lvl_3,description,price,delivery_type,clarity,conciseness
0,my,AE068HBAA3RPRDANMY,Cuba Heartbreaker Eau De Parfum Spray 100ml/3.3oz,Health & Beauty,Bath & Body,Hand & Foot Care,Formulated with oil-free hydrating botanicals/...,128.00,international,1,1
1,my,AN680ELAA9VN57ANMY,Andoer 150cm Cellphone Smartphone Mini Dual-He...,"TV, Audio / Video, Gaming & Wearables",Audio,Live Sound & Stage,<ul> <li>150cm mini microphone compatible for ...,25.07,international,1,1
2,my,AN957HBAAAHDF4ANMY,ANMYNA Complaint Silky Set 柔顺洗发配套 (Shampoo 520...,Health & Beauty,Hair Care,Shampoos & Conditioners,<ul> <li>ANMYNA Complaint Silky Set (Shampoo 5...,118.00,local,1,0
3,my,AR511HBAXNWAANMY,Argital Argiltubo Green Clay For Face and Body...,Health & Beauty,Men's Care,Body and Skin Care,<ul> <li>100% Authentic</li> <li>Rrefresh and ...,114.80,international,1,1
4,my,AS575ELCMZ4WANMY,Asus TP300LJ-DW004H Transformer Book Flip 4GB ...,Computers & Laptops,Laptops,Traditional Laptops,"<div class=""prod_content""> <div class=""prod_de...",2599.00,local,1,1


### Loading validation data

In [6]:
#Loading the main data
data_val = pd.read_csv(r'C:\Users\aarib\6thSemester\IR\IRProject\Product Title Classification\CIKMAnalytiCup2017_Lazada\validation\data_valid.csv')

#Loading labels
clarity_labels = pd.read_csv(r'C:\Users\aarib\6thSemester\IR\IRProject\Product Title Classification\CIKMAnalytiCup2017_Lazada\validation\clarity_valid.predict', header=None)
conciseness_labels = pd.read_csv(r'C:\Users\aarib\6thSemester\IR\IRProject\Product Title Classification\CIKMAnalytiCup2017_Lazada\validation\conciseness_valid.predict', header=None)

#Assigning meaningful column names
clarity_labels.columns = ['clarity']
conciseness_labels.columns = ['conciseness']

#Combining everything into one DataFrame
validation = pd.concat([data_val, clarity_labels, conciseness_labels], axis=1)

print("Shape of validation data:", validation.shape)
validation.head()

Shape of validation data: (11838, 11)


,my,AP564ELASSTWANMY,Apple MacBook Pro MGXC2ZP/A 16GB i7 15.4-inch Retina Display Laptop,Computers & Laptops,Laptops,Macbooks,OS X Lion<br> Intel Core i7<br> 15-inch Retina Display<br> 16GB RAM / 512GB Flash<br> GeForce GT 750M<br> WiFi + BT4 + RJ45<br>,12550.0,local,clarity,conciseness
0,my,BR924HBAA5B3TLANMY,BRAND'S® American Ginseng Triple Pack (3x 6's)...,Health & Beauty,Food Supplements,Well Being,<ul> <li>Traditionally used to calm the mind a...,105.0,local,0.34139,0.98747
1,my,CA673ELAA5UG3XANMY,Canon EOS M10 Mirrorless Digital Camera 18MP w...,Cameras,Mirrorless,NaN,<div> <ul> <li>18.0MP APS-C CMOS Sensor</li> <...,1588.0,local,0.22758,0.29770
2,my,DE759ELAA7QM1XANMY,"Dell LED Monitor 23"" (E2316H)",Computers & Laptops,Computer Accessories,Monitors,"<div class=""prod_content""> <div class=""prod_de...",565.0,local,0.36162,0.59796
3,my,ES802OTAABHAY8ANMY,Esprit Tallac Brave Nubuck Sand ES107601001 Be...,Watches Sunglasses Jewellery,Watches,Men,<ul> <li>stainless steel case</li> <li>mineral...,279.0,local,0.83418,0.56371
4,my,HP961ELAABF7N7ANMY,"(Refurbished) HP Compaq 3330 Pro MT + 19"" LCD",Computers & Laptops,Desktops Computers,All-purpose,"<ul> <li>Model : HP Compaq 3330 Pro MT + 19"" L...",1259.0,local,0.12018,0.50220


### Adding column names to validation data

In [7]:
validation.columns = [
    'country', 'product_id', 'title', 
    'category_lvl_1', 'category_lvl_2', 'category_lvl_3',
    'description', 'price', 'delivery_type', 
    'clarity', 'conciseness'
]

In [8]:
validation.head()

,country,product_id,title,category_lvl_1,category_lvl_2,category_lvl_3,description,price,delivery_type,clarity,conciseness
0,my,BR924HBAA5B3TLANMY,BRAND'S® American Ginseng Triple Pack (3x 6's)...,Health & Beauty,Food Supplements,Well Being,<ul> <li>Traditionally used to calm the mind a...,105.0,local,0.34139,0.98747
1,my,CA673ELAA5UG3XANMY,Canon EOS M10 Mirrorless Digital Camera 18MP w...,Cameras,Mirrorless,NaN,<div> <ul> <li>18.0MP APS-C CMOS Sensor</li> <...,1588.0,local,0.22758,0.29770
2,my,DE759ELAA7QM1XANMY,"Dell LED Monitor 23"" (E2316H)",Computers & Laptops,Computer Accessories,Monitors,"<div class=""prod_content""> <div class=""prod_de...",565.0,local,0.36162,0.59796
3,my,ES802OTAABHAY8ANMY,Esprit Tallac Brave Nubuck Sand ES107601001 Be...,Watches Sunglasses Jewellery,Watches,Men,<ul> <li>stainless steel case</li> <li>mineral...,279.0,local,0.83418,0.56371
4,my,HP961ELAABF7N7ANMY,"(Refurbished) HP Compaq 3330 Pro MT + 19"" LCD",Computers & Laptops,Desktops Computers,All-purpose,"<ul> <li>Model : HP Compaq 3330 Pro MT + 19"" L...",1259.0,local,0.12018,0.50220


### Text Cleaning & Vectorization

A custom function clean_text():

Removes HTML tags using BeautifulSoup

Eliminates special characters and digits

Converts text to lowercase

Strips extra whitespace

This function is applied to both the product title and description for the training and validation sets.

The cleaned title and description are concatenated to form a new feature called text.

Then:

A character-level n-gram vectorizer is defined using CountVectorizer with ngram_range=(2, 6) and max_features=5000, as outlined in the original paper.

The vectorizer is fitted on the training text, and both training and validation texts are transformed into numerical feature vectors.

Finally, the target variables y_clarity and y_conciseness are extracted for model training.

In [9]:
def clean_text(text):
    if pd.isna(text):
        return ""
    #Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    #Remove special characters and digits
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    #Convert to lowercase
    text = text.lower()
    #Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

#Applying transformations  to both train and validation sets
for df in [train, validation]:
    df['clean_title'] = df['title'].apply(clean_text)
    df['clean_desc'] = df['description'].apply(clean_text)
    df['text'] = df['clean_title'] + " " + df['clean_desc']

In [10]:
#Using character n-grams as described in the paper (ngram_range = (2,6))
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 6), max_features=5000)

#Fitting on train text, transform train and validation
X_train = char_vectorizer.fit_transform(train['text'])
X_val = char_vectorizer.transform(validation['text'])

#Targets
y_clarity = train['clarity']
y_conciseness = train['conciseness']

### Bagged Ensemble Model Training with LightGBM

This section implements a robust ensemble strategy to improve prediction accuracy by reducing variance:

Model Choice:

Used LightGBM Regressor (LGBMRegressor) for its speed and performance on high-dimensional, sparse data (like character n-grams).

Ensemble Strategy – Bagging + K-Fold CV:

Bagging (n_bags=4): Run the full 10-fold cross-validation process four times with different random seeds to simulate different “bags” of models.

K-Fold Cross-Validation (n_folds=10): Each bag uses 10-fold CV to split the training data into training and validation subsets.

In total, 40 models are trained (4 bags × 10 folds).

Predictions from all models are averaged to produce final validation predictions.

Model Configuration:

n_estimators=300: Each model trains up to 300 boosting iterations.

learning_rate=0.05: Slower learning for better generalization.

num_leaves=31: Controls model complexity.

random_state: Varied per bag to encourage diversity in learned models.

Data Compatibility Fix:

Transformed X_train and X_val to np.float32 to resolve type compatibility errors with LightGBM (which expects float32 or float64 input, not int64).

Target-wise Training:

The process is run independently for both target variables:

clarity

conciseness

This setup helps create more stable and generalizable predictions by aggregating results from a diverse set of models trained on different subsets of data.

In [12]:
def run_lgbm_bagged(X, y, X_val, n_bags=4, n_folds=10):
    val_preds = np.zeros(X_val.shape[0])
    
    for bag in range(n_bags):
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=42 + bag)
        for train_idx, valid_idx in tqdm(kf.split(X), desc=f"Bag {bag+1}/{n_bags}"):
            X_train_kf, X_valid_kf = X[train_idx], X[valid_idx]
            y_train_kf, y_valid_kf = y.iloc[train_idx], y.iloc[valid_idx]
            
            model = lgb.LGBMRegressor(
                n_estimators=300,
                learning_rate=0.05,
                max_depth=-1,
                num_leaves=31,
                random_state=42
            )
            model.fit(X_train_kf, y_train_kf)
            val_preds += model.predict(X_val) / (n_folds * n_bags)
    
    return val_preds


# Fix dtype for LightGBM
X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)

# Run for both targets
print("Training ensemble for CLARITY...")
val_preds_clarity = run_lgbm_bagged(X_train, y_clarity, X_val)

print("\nTraining ensemble for CONCISENESS...")
val_preds_conciseness = run_lgbm_bagged(X_train, y_conciseness, X_val)

Training ensemble for CLARITY...


Bag 1/4: 0it [00:00, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.407419 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36642
[LightGBM] [Info] Number of data points in the train set: 32654, number of used features: 5000
[LightGBM] [Info] Start training from score 0.943315


Bag 1/4: 1it [00:29, 29.80s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.463508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 36547
[LightGBM] [Info] Number of data points in the train set: 32654, number of used features: 5000
[LightGBM] [Info] Start training from score 0.943498


...

Bag 4/4: 8it [01:39, 12.99s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.491581 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36611
[LightGBM] [Info] Number of data points in the train set: 32655, number of used features: 5000
[LightGBM] [Info] Start training from score 0.686357


Bag 4/4: 9it [01:51, 12.54s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.586812 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36587
[LightGBM] [Info] Number of data points in the train set: 32655, number of used features: 5000
[LightGBM] [Info] Start training from score 0.685439


Bag 4/4: 10it [02:03, 12.31s/it]


In [14]:
#Extracting actual labels from validation set
y_val = validation[['clarity', 'conciseness']]

#Calculate RMSE
rmse_clarity = mean_squared_error(y_val['clarity'], val_preds_clarity, squared=False)
rmse_conciseness = mean_squared_error(y_val['conciseness'], val_preds_conciseness, squared=False)

print(f"Validation RMSE - Clarity: {rmse_clarity:.5f}")
print(f"Validation RMSE - Conciseness: {rmse_conciseness:.5f}")

Validation RMSE - Clarity: 0.52931
Validation RMSE - Conciseness: 0.35273


### Results Comparison: Our Implementation vs. Research Paper

This section compares our model’s validation performance with the results reported in the CIKM AnalytiCup 2017 paper titled "Lazada Product Title
Quality Challenge Bagging Model for Product Title Quality with Noise".

Target	     Algorithm	   Paper RMSE	  Our RMSE
Conciseness	 XGBoost	      0.31553	   0.35273
Clarity	     XGBoost	      0.20745	   0.52931

Observations:
Our RMSE values are higher, particularly for the Clarity task.

The paper’s results were achieved using additional techniques such as:

Ensemble of different algorithms (e.g., XGBoost, Ridge, SVR)

More aggressive hyperparameter tuning

Possibly more preprocessing steps or feature engineering

Our pipeline is simpler (using only LightGBM + character n-grams), but still reasonably competitive for conciseness.